In [2]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [1]:
1+2

3

# Getting sentence classifications

In [3]:
f = open("LitCovid_sent_classification_formatted.sents", "r")
sec_class_sents = f.read()
f = open("LitCovid_sent_classification_formatted.txt", "r")
sec_class_txt = f.read()

In [4]:
sec_pmids[6469]

NameError: name 'sec_pmids' is not defined

In [8]:
sec_classes = sec_class_sents.split("\n\n\n")
sec_pmids = [t.split(" | ")[0] for t in sec_class_txt.split("\n")]
sections = {}
for i in range(len(sec_pmids)):
    if sec_pmids[i] != "":
        sections[sec_pmids[i]] = [(t.split("||")[0], t.split("||")[1]) for t in sec_classes[i].split("\n")]

In [9]:
split_sections = {}
for i in tqdm(range(len(sec_pmids))):
    if sec_pmids[i] != "":
        sect = sections[sec_pmids[i]]
        temp_sect = []
        for line in sect:
            if len(line[1].split()) > 30:
                text = line[1].split()
#                 print(text)
                split_sentences = [' '.join(text[i:i+30]) for i in range(0, len(text),30)]
#                 print(split_sentences)
                for sent in split_sentences:
                    temp_sect.append((line[0], sent))
            else:
                temp_sect.append(line)
        
        split_sections[sec_pmids[i]] = temp_sect

100%|███████████████████████████████████████████████████████████████████████████| 6470/6470 [00:00<00:00, 12750.56it/s]


In [135]:
len("Gaining a deeper understanding of the interaction between Coronaviruses and the innate immune systems of the hosts may shed light on the development and persistence of inflammation in the lungs".split())

30

In [136]:
# sec_classes[i].split("\n")
sections.get("31981224")

[('TITLE', 'Coronavirus infections and immune responses'),
 ('BACKGROUND',
  'Coronaviruses ( CoVs ) are by far the largest group of known positive - sense RNA viruses having an extensive range of natural hosts .'),
 ('BACKGROUND',
  'In the past few decades , newly evolved Coronaviruses have posed a global threat to public health .'),
 ('BACKGROUND',
  'The immune response is essential to control and eliminate CoV infections , however , maladjusted immune responses may result in immunopathology and impaired pulmonary gas exchange .'),
 ('BACKGROUND',
  'Gaining a deeper understanding of the interaction between Coronaviruses and the innate immune systems of the hosts may shed light on the development and persistence of inflammation in the lungs and hopefully can reduce the risk of lung inflammation caused by CoVs .'),
 ('CONCLUSIONS',
  'In this review , we provide an update on CoV infections and relevant diseases , particularly the host defense against CoV - induced inflammation of lu

In [23]:
custom_syns = [['COVID-19', "SARS-CoV-2", "SARS-CoV-2 infection", "COVID-19 infection", 
  "coronavirus disease 2019 ( COVID-19 )", 'Covid-19', 'laboratory-confirmed COVID-19', 
  "coronavirus disease 2019", "COVID-19 disease", "coronavirus disease", "SARS-CoV", 
  "COVID-19 positive", "SARS-COV-2", "SARS-CoV-2 infections",  "Covid-19 infection", 
  "COVID-19 infected", "COVID-19-positive", "SARS-CoV-2 infected", "SARS-CoV-2-infected",
  "2019-nCoV", "COVID-19 infections", "COVID 19", "COVID19", "Covid-19 infection",
  "severe acute respiratory syndrome coronavirus 2 ( SARS-CoV-2 ) infection", 
  'coronavirus disease ( COVID-19 )', "CoV infections", "coronavirus", "2019-new coronavirus",
  "SARS-CoV-2 virus", "coronavirus infections"],
 ["severe COVID-19", "severe and critical COVID-19", "severe COVID-19 infection", 
  'severe coronavirus disease 2019 ( COVID-19 )',
  'severe coronavirus disease 2019 ( COVID-19 )', 
  'severe acute respiratory syndrome ( SARS )', 'severe and critical COVID-19',
  'severe acute respiratory syndrome coronavirus 2',
  'critically ill COVID-19', "severe COVID-19 disease"],
 ["severe COVID-19 pneumonia", "severe pneumonia", "COVID-19 pneumonia", "SARS-CoV-2 pneumonia",
  'novel coronavirus pneumonia', "2019-nCoV pneumonia"],
 ["male", "men", "males", "male sex"],
 ["infection", "infected"],
 ["female", "women", "females", "female sex"],
 ["severe", "severe disease", "severe cases", "severely ill", "severe illness"],
 ["critical", "critically ill",  "critical illness"],
 ["comorbidity", "comorbidities"],
 ["ICU", "intensive care unit ( ICU )", "intensive care"],
 ["diabetes", "diabetes mellitus", "diabetic"],
 ["elderly", "older age", "older adults", "older"],
 ["hydroxychloroquine", "chloroquine"],
 ["dead", "deceased", "mortality", "died", "death"]]

lower_syns = []
for syn in custom_syns:
    lower_syns.append([s.lower() for s in syn])

    
def get_syn(ent):
    covid_syns = ["covid", "sars-cov", "corona", "ncov", "cov infections"]
    severe_syns = ["severe"]
    if any(word in ent.lower() for word in covid_syns) and any(word in ent.lower() for word in severe_syns):
        return "severe covid-19"
    elif any(word in ent.lower() for word in covid_syns) and "pneumonia" in ent.lower():
        return "COVID-19 pneumonia"
#     elif any(word in ent.lower() for word in severe_syns):
#         return "severe"
    else:
        for l in lower_syns:
            if ent.lower() in l:

                return l[0]
        
    return ent

# Get data

In [24]:

folder = "../LitCovid_Treatment_json/"
dirs = os.listdir(folder)
pmids = []
entities = []

singulars = 0
for file in tqdm(dirs, desc = "dirs"):
# for file in dirs[:3]:
#     try:
    if file not in ["exceptionlist.txt", "label2id.pkl", "predict.tf_record"]:

        #get filepath and read in data
        file_path = os.path.join(folder, file)
        temp = pd.read_json(file_path)
        pmid = file[:-5]
        section = split_sections.get(pmid)

#             if section[0][0] == "TITLE":
#                 sec_idx = 1
#             else:
#                 sec_idx = 0
#         print(temp)
#         print(section)
        for i in range(temp.shape[0]):

            #get entities sentence by sentence
            ents_dict = temp.sentences.values[i].get("entities")
            orig_text = temp.sentences.values[i].get("text")

            if i < len(section):
                temp_sec = section[i][0]
#                     sec_idx +=1

            else:
                temp_sec = "ERROR"

            if len(ents_dict.keys()) > 0:
                temp_ents = [(ents_dict[k]["text"], ents_dict[k]["class"], ents_dict[k]["negation"], orig_text, temp_sec, pmid) for k in ents_dict.keys()]
                true_ents = []
                for i in range(len(temp_ents)):
                    ent = temp_ents[i][0]
                    cl = temp_ents[i][1]
                    neg = temp_ents[i][2]
                    added = False

                    if len(ent) > 1: #make sure it's not punctuation

                        if len(ent.split()) > 1: #if phrase

                            #check to see if it's in custom set of synonyms
                            new_ent = get_syn(ent)
                            true_ents.append((new_ent, cl, neg, orig_text, temp_sec, pmid))

                        else: #if single word
                            if ent.lower() not in stop_words: #if not stop word

                                #check to see if it's in custom set of synonyms
                                new_ent = get_syn(ent)

                                if new_ent != ent:
                                    true_ents.append((new_ent, cl, neg, orig_text, temp_sec, pmid))

                                else:
                                    ent = singularize(ent)
                                    new_ent = get_syn(ent)
                                    true_ents.append((new_ent, cl, neg, orig_text, temp_sec, pmid))       

                                    
                entities += true_ents
#     except:
#         print(pmid)
#         print(section)
#         print(temp.sentences.values)


dirs:   2%|█▏                                                                      | 105/6459 [00:00<00:38, 164.07it/s]

critical
critical
critical
critical
critical
critical


dirs:   2%|█▋                                                                      | 150/6459 [00:00<00:47, 133.28it/s]

critical
critical
critical
critical


dirs:   4%|███                                                                     | 277/6459 [00:01<00:41, 150.19it/s]

...

dirs:  99%|██████████████████████████████████████████████████████████████████████▏| 6383/6459 [00:51<00:00, 114.86it/s]

critical
critical


dirs: 100%|██████████████████████████████████████████████████████████████████████▉| 6448/6459 [00:51<00:00, 121.82it/s]

critical
critical


dirs: 100%|███████████████████████████████████████████████████████████████████████| 6459/6459 [00:51<00:00, 124.92it/s]


In [26]:
ent_df = pd.DataFrame(entities, columns=["text", "type", "negation", "sentence", "section", "pmid"])
ent_df.to_csv("LitCovid_treatment_ents.csv")
ent_df


,text,type,negation,sentence,section,pmid
0,covid-19,Participant,0,"In this review , we provide an update on CoV i...",BACKGROUND,31981224
1,relevant,Participant,0,"In this review , we provide an update on CoV i...",BACKGROUND,31981224
2,host defense against,Outcome,0,"In this review , we provide an update on CoV i...",BACKGROUND,31981224
3,covid-19,Participant,0,This highlights the importance of immune respo...,CONCLUSIONS,31981224
4,covid-19,Participant,0,"As of January 22 , 2020 , a total of 571 cases...",TITLE,31996494
5,inflammation,Outcome,0,"Here , we propose for the first time that infl...",RESULTS,32013309
6,covid-19,Participant,0,"Here , we propose for the first time that infl...",RESULTS,32013309
7,inhibited,modifier,0,"Here , we propose for the first time that infl...",RESULTS,32013309
8,accelerate,modifier,0,accelerate its control .,BACKGROUND,32015560
9,control,Outcome,0,accelerate its control .,BACKGROUND,32015560


In [140]:
file = "32013309.json"
file_path = os.path.join(folder, file)
temp = pd.read_json(file_path)
    
temp.sentences.values

array([{'Section': 'METHODS', 'text': 'Coronavirus can cause respiratory syndrome which to date has affected about twelve thousand individuals , especially in China .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Coronavirus is interspecies and can also be transmitted from man to man , with an incubation ranging from 1 to 14 days .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Human coronavirus infections can induce not only mild to severe respiratory diseases , but also inflammation , high fever , cough , acute respiratory tract infection and dysfunction of internal', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'organs that may lead to death .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Coronavirus infection ( regardless of the various types of corona virus ) is primarily attacked by immune cells including mast cells ( MCs ) , which are located in the', 'entities': {}, 

In [141]:
sections.get("32013309")

[('TITLE',
  'Mast cells contribute to coronavirus-induced inflammation: new anti-inflammatory strategy.'),
 ('BACKGROUND',
  'Coronavirus can cause respiratory syndrome which to date has affected about twelve thousand individuals , especially in China .'),
 ('RESULTS',
  'Coronavirus is interspecies and can also be transmitted from man to man , with an incubation ranging from 1 to 14 days .'),
 ('RESULTS',
  'Human coronavirus infections can induce not only mild to severe respiratory diseases , but also inflammation , high fever , cough , acute respiratory tract infection and dysfunction of internal organs that may lead to death .'),
 ('RESULTS',
  'Coronavirus infection ( regardless of the various types of corona virus ) is primarily attacked by immune cells including mast cells ( MCs ) , which are located in the submucosa of the respiratory tract and in the nasal cavity and represent a barrier of protection against microorganisms .'),
 ('RESULTS',
  'Viral activate MCs release early

# Send counters to pickle file

In [142]:
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
p_count = Counter(ent_df[ent_df.type == "Participant"].text.values)
i_count = Counter(ent_df[ent_df.type == "Intervention"].text.values)
o_count = Counter(ent_df[ent_df.type == "Outcome"].text.values)

counters = [p_count, i_count, o_count]
c_names = ["p_count", "i_count", "o_count"]
for i in range(len(counters)):
    with open("Pickles/" + c_names[i] + '.pickle', 'wb') as file:
        pickle.dump(counters[i], file)
    

# Building weighted graph

In [ ]:
from collections import Counter
import itertools as it

# get data and group
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
g = ent_df.groupby(["pmid", "type"])

#initialize variables
groups = list(g.groups)
types = ["Participant", "Outcome", "Intervention"]
edge_weight = Counter()
nodes = []

for p in tqdm(ent_df.pmid.values):
    temp = []
    for tup in list(it.product([p], types)):
        if tup in groups:
            
            #get pmid entities
            grp = g.get_group(tup)
            
            #format appropriately
            temp_nodes = ["{} ({})".format(t, tup[1][0]) for t in grp.text.values]
            temp.append(temp_nodes)
            nodes += temp_nodes
        else:
            temp.append([])
            
    #get edges
    products = [list(it.product(temp[0], temp[1])), list(it.product(temp[0], temp[2])), list(it.product(temp[1], temp[2]))]
    
    #add edges to adj list
    for prod in products:
        if prod != []:
            edge_weight.update(prod)

 28%|█████████████████▊                                              | 33980/121669 [03:07<12:23, 117.86it/s]

In [ ]:
with open("Pickles/" + 'wb') as file:
    pickle.dump(edge_weight, file)

In [ ]:
with open('edges.pickle', 'rb') as file:
    edge_weight = pickle.load(file)

In [ ]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

G = nx.Graph()
for item in edge_weight.most_common(200):
    G.add_edge(item[0][0], item[0][1], weight=item[1]*100)



g = Network(height=800, width=800, notebook=True)
g.barnes_hut()
g.from_nx(G)
g.show_buttons(filter_=['physics'])
g.show("ex.html")

